Hw7 - Fatima Abdurrahman

Note - I only got through points 1-3 on the homework pdf.

In [1]:
from PIL import Image
import numpy as np
from skimage import feature, data
from skimage.feature import blob_dog, blob_log, blob_doh
import matplotlib.pyplot as plt
%matplotlib inline
from os import listdir
from os.path import isfile, join
from sklearn.ensemble import RandomForestClassifier
import sys

In [26]:
def features(image):
    
    ## Returns 15 real numbered features for an input image

    I = np.asarray(Image.open(image))

    ## Feature 1: image size

    size = np.size(I) #1 - Array size

    ## Features 2-9: averages, maxes, and ratios of colors in different channels

    aveR = np.average(I[:,:,0]) #2 - Average of red channel
    aveG = np.average(I[:,:,1]) #3 - Average of green channel
    aveB = np.average(I[:,:,2]) #4 - Average of blue channel
    maxR = np.max(I[:,:,0]) #5 - Max of red channel
    maxG = np.max(I[:,:,1]) #6 - Max of green channel
    maxB = np.max(I[:,:,2]) #7 - Max of blue channel
    R_G = aveR/aveG #8 - Ratio of mean red to mean green
    R_B = aveR/aveB #9 - Ratio of mean red to mean blue

    ## Features 10-12: Edges - Max row and column of outline image, fit horizontal line

    chan_ave = np.average(I, axis=2)
    edges1 = feature.canny(chan_ave, sigma=3)
    vert_max = np.argmax(np.sum(edges1, axis=0)) #10 Max column of outline image
    horz_max = np.argmax(np.sum(edges1, axis=1)) #11 Max row of outline image

    edges1 = edges1.astype(int)
    x = []
    y = []
    count = 0
    for i in range(np.shape(edges1)[0]):
        for j in range(np.shape(edges1)[1]):
            if edges1[i][j] == 1:
                y.append(i)
                x.append(j)
                count += 1
    x.reverse()
    fity = np.polyfit(x, y, 0)[0] #12 Y of best fit horizontal line

    ## Featres 13 - 15: Blob detection - number, size, 1 position

    blobs_doh = blob_doh(chan_ave, max_sigma=30, threshold=0.01)
    num_blobs = np.shape(blobs_doh)[0] #13 Number of blobs (hessian)
    blob_size = np.average(blobs_doh, axis=0)[2] #14 Average blob radius
    blob1dist = ((blobs_doh[0][0]-(np.shape(chan_ave)[1]/2))**2 + \
                 (blobs_doh[0][1]-(np.shape(chan_ave)[0]/2))**2)**0.5 #15 Distance from blob 1 to center

    features = [size, aveR, aveG, aveB, maxR, maxG, maxB, R_G, R_B, vert_max, horz_max, \
               fity, num_blobs, blob_size, blob1dist]
    
    return features

In [27]:
#makes array of all images, runs the features function on them all (x), and 
#makes an array of each x elements category name (y)

categories = listdir("50_categories")[1::]
dat = []
y = []
for i in range(len(categories)): 
    first5  = listdir("50_categories/"+categories[i])
    name = first5[0].split("_")[0]
    for file in first5:
        path = "50_categories/"+name+"/"+first5[i]
        dat.append(path)
        y.append(name)
[dat, y]

x = []
dat
for i in range(len(dat)):
    data = features(dat[i])
    x.append(data)


In [ ]:
#Organize data into training (90%) and testing (10%)

split = int(len(y)*0.9)
Xtr = x[:split]
Ytr = y[:split]
Xte = x[split:]
Yte = y[split:]

classifier = RandomForestClassifier(n_estimators=30)
classifier.fit(Xtr, Ytr)
pred_rf = classifier.predict(Xte) 

print("True Class / Predicted class")
print(np.vstack((Yte[0:10],pred_rf[0:10])))